In [11]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

ЗДЕСЬ И СОБЫТИЯ ЯНДЕКСА И РОССТАТ

In [ ]:
events1 = pd.read_csv('события.csv')
events2 = pd.read_csv('росстат_общий_new1.csv')


df = events2.merge(
    events1[['Event Text', 'region_name', 'municipality', 'year']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df.to_csv('общие_данные_2010t+.csv', index=False)
df

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Земля,Расходы,ПотреблениеВ,ПотреблениеТ,ПотреблениеЭ,БезДорог,Прибыль,Бюджет,Квартиры,Зарплата,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-433.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-375.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,345.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519659,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
519660,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
519661,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
519662,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ОБЪЕДИНЕНИЕ ПРОТЕСТОВ С ДАННЫМИ РОССТАТА 

1. ВООБЩЕ 
2. ТОЛЬКО ЗА 2012

In [15]:
data = pd.read_csv('общие_данные_2010t+.csv')

protest_events = ["Demonstrate or rally", "Criticize or denounce", "Defy norms, law"]
protests = data[data['Event Text'].isin(protest_events)]
protests.to_csv('протесты_2010t+.csv', index=False)

protests1 = protests[protests['year'].isin([2009, 2010])].reset_index(drop=True)
protests1 = protests1.drop_duplicates(subset=['oktmo']).reset_index(drop=True)

protests1.to_csv('протесты2009_2010t+.csv', index=False)
protests

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Земля,Расходы,ПотреблениеВ,ПотреблениеТ,ПотреблениеЭ,БезДорог,Прибыль,Бюджет,Квартиры,Зарплата,Event Text
245,2006,Татарстан,Агрызский,92601000,0.0,0.0,0.0,0.0,0.0,179662.2,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,Criticize or denounce
246,2006,Татарстан,Агрызский,92601000,0.0,0.0,0.0,0.0,0.0,179662.2,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,Criticize or denounce
282,2006,Татарстан,Агрызский,92601000,0.0,0.0,0.0,0.0,0.0,179662.2,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,Criticize or denounce
294,2006,Татарстан,Агрызский,92601000,0.0,0.0,0.0,0.0,0.0,179662.2,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,Demonstrate or rally
348,2006,Татарстан,Агрызский,92601000,0.0,0.0,0.0,0.0,0.0,179662.2,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,Criticize or denounce
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504660,2023,Хабаровский,Хабаровский,8655000,0.0,61.9,0.0,0.0,82008.0,0.0,0.0,18.7,0.22,1333.50,2.71,0.0,0.0,0.0,67779.7,Demonstrate or rally
504662,2023,Хабаровский,Хабаровский,8655000,0.0,61.9,0.0,0.0,82008.0,0.0,0.0,18.7,0.22,1333.50,2.71,0.0,0.0,0.0,67779.7,Demonstrate or rally
504665,2023,Хабаровский,Хабаровский,8655000,0.0,61.9,0.0,0.0,82008.0,0.0,0.0,18.7,0.22,1333.50,2.71,0.0,0.0,0.0,67779.7,Demonstrate or rally
504667,2023,Хабаровский,Хабаровский,8655000,0.0,61.9,0.0,0.0,82008.0,0.0,0.0,18.7,0.22,1333.50,2.71,0.0,0.0,0.0,67779.7,Demonstrate or rally


ЕДИНИЦЫ БЕЗ ПРОТЕСТОВ СРЕДИ ВСЕХ СОБЫТИЙ ЗА ВСЕ ВРЕМЯ

In [ ]:
lol = pd.read_csv("протесты_2010t+.csv") 
sobytia = pd.read_csv("общие_данные_2010t+.csv")


noprotestALL = sobytia[~sobytia['municipality'].isin(lol['municipality'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2010t+.csv', index=False)
noprotestALL

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Земля,Расходы,ПотреблениеВ,ПотреблениеТ,ПотреблениеЭ,БезДорог,Прибыль,Бюджет,Квартиры,Зарплата,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-433.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,284.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-375.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,345.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418325,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
418326,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
418327,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
418328,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ПОИСК КОНТРОЛЬНОЙ ГРУППЫ (НЕПРОТЕСТНЫЕ ЕДИНИЦЫ ЗА 2011 ИЗ ТЕХ ЧТО ПРОТЕСТНЫЕ В 2012)

In [17]:
lol = pd.read_csv("протесты2009_2010t+.csv") 
sobytia = pd.read_csv("общие_данные_2010t+.csv")

df_2011 = sobytia[sobytia['year'] == 2008]


filtered_df = df_2011[df_2011['oktmo'].isin(lol['oktmo'])]

pattern = "Demonstrate or rally|Criticize or denounce|Defy norms, law"
mask = filtered_df['Event Text'].str.contains(pattern, case=False, na=False)
municipalities_with_tag = filtered_df[mask]['oktmo'].unique()


final_df = filtered_df[~filtered_df['oktmo'].isin(municipalities_with_tag)].reset_index(drop=True)

final_df = final_df.drop_duplicates(subset='oktmo').reset_index(drop=True)
final_df.to_csv('без_протестов2008_2010t+.csv', index=False)
final_df



,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Земля,Расходы,ПотреблениеВ,ПотреблениеТ,ПотреблениеЭ,БезДорог,Прибыль,Бюджет,Квартиры,Зарплата,Event Text
0,2008,Хакасия,Абакан,95701000,21.1,0.00,381.0,6300.0,0.0,11238.0,5930737.0,0.0,0.0,0.0,0.0,0.0,-13052.0,411.00,18275.2,"Arrest, detain, or charge with legal action"
1,2008,Ставропольский,Александровский,7602000,0.0,0.00,0.0,0.0,0.0,0.0,1110569.0,0.0,0.0,0.0,0.0,0.0,6993.0,359.48,7463.3,NaN
2,2008,Ставропольский,Александровский,7602402,0.0,1.46,0.0,0.0,0.0,37450.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,7866.5,NaN
3,2008,Краснодарский,Анапа,3703000,1.0,11.30,1600.0,34000.0,0.0,98186.0,6423662.7,0.0,0.0,0.0,0.0,0.0,-41505.0,360.00,12470.0,"Arrest, detain, or charge with legal action"
4,2008,Архангельская,Архангельск,11701000,474.7,0.00,3065.0,3181.0,0.0,29445.0,17943657.8,0.0,0.0,0.0,0.0,0.0,-44553.0,0.00,21012.6,Make an appeal or request
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,2008,Дагестан,Хасавюрт,82735000,358.2,0.00,4600.0,4600.0,0.0,3848.0,1947649.0,0.0,0.0,0.0,0.0,0.0,22137.0,0.00,8241.9,Make statement
78,2008,Ростовская,Целинский,60656000,12.1,4.11,0.0,6500.0,0.0,212960.0,1143971.0,0.0,0.0,0.0,0.0,0.0,9307.0,416.28,8707.9,NaN
79,2008,Тульская,Чернский,70646000,0.0,0.00,0.0,14700.0,0.0,161429.0,711542.0,0.0,0.0,0.0,0.0,0.0,9451.0,533.64,8898.8,Investigate
80,2008,Адыгея,Шовгеновский,79640000,0.0,0.00,0.0,0.0,0.0,0.0,448370.0,0.0,0.0,0.0,12.0,0.0,2752.0,312.80,6265.3,fight with small arms and light weapons


Создание регрессии

In [ ]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

df_protests = pd.read_csv("протесты2009_2010t+.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t+.csv")
df_events_full = pd.read_csv("общие_данные_2010t+.csv")

required_years = {2008.0, 2011.0, 2012.0, 2013.0}

df_events = df_events[df_events['year'].isin([2009, 2010])].reset_index(drop=True)

features = [
            'ПотреблениеЭ', 
            'ПотреблениеВ', 
            'Бюджет',
            'БезДорог', 
            'Земля',
            'Квартиры',  
            
            # 'Население', 
            # 'Прибыль',
            # 'Зарплата',
            # 'Расходы',

            # 'СреднегодН',
            # 'ПотреблениеТ', 
            # 'ЭконАктив'
            # 'Прирост', 
            # 'Рождаемость',
            # 'Смертность',
            # 'Родившихся',
            # 'Инвестиции',
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)


exclude_dict = {
    'Абакан': ['Усть-Абаканский', 'Алтайский', 'Минусинский', 'Минусинск', 'Черногорск'],
    'Анапа': ['Темрюкский', 'Крымский', 'Новороссийск'],
    'Архангельск': ['Северодвинск', 'Приморский'],
    'Архаринский': ['Облученский', 'Верхнебуреинский', 'Бурейский'],
    'Бабаюрт': ['Шелковской', 'Хасавюртовский', 'Кизилюртовский', 'Кумторкалинский', 'Кизлярский'],
    'Балашиха': ['Королев', 'Люберцы', 'Богородский', 'Щелково', 'Раменский'],
    'Благовещенск': ['Благовещенский', 'Кушнаревский', 'Имангуловский'],
    'Бокситогорский': ['Тихвинский', 'Любытинский', 'Хвойнинский', 'Чагодощенский', 'Бабаевский'],
    'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
    'Варгашинский': ['Кетовский', 'Курган', 'Половинский', 'Лебяжьевский', 'Мокроусовский', 'Упоровский', 'Белозерский'],
    'Вологда': ['Вологодский'],
    'Воронеж': ['Хохольский', 'Семилукский', 'Рамонский', 'Новоусманский', 'Каширский'],
    'Гатчинский': ['Лужский', 'Волосовский', 'Тосненский', 'Пушкин', 'Ломоносовский'],
    'Геленджик': ['Туапсинский', 'Северский', 'Абинский', 'Крымский', 'Новороссийск'],
    'Горно-Алтайск': ['Майминский'],
    'Грязовецкий': ['Вологодский', 'Междуреченский', 'Солигаличский', 'Буйский', 'Любимский', 'Первомайский', 'Пошехонский'],
    'Гулькевичский': ['Кавказский', 'Новоалександровский', 'Новокубанский', 'Курганинский', 'Тбилисский'],
    'Гурьевский': ['Зеленоградский', 'Калининград', 'Полесский', 'Гвардейский', 'Багратионовский', 'Ладушкинский'],
    'Добрянский': ['Краснокамский', 'Ильинский', 'Юсьвинский', 'Березники', 'Александровский', 'Губаха', 'Чусовской', 'Пермский'],
    'Домодедово': ['Подольск', 'Ленинский', 'Раменский', 'Чехов', 'Ступино', 'Щербинка'],
    'Дубна': ['Кимрский', 'Кимры', 'Талдомский', 'Дмитровский', 'Конаковский'],
    'Завьяловский': ['Ижевск', 'Увинский', 'Малопургинский', 'Сарапульский', 'Чайковский', 'Воткинский', 'Якшур-Бодьинский'],
    'Ивановский': ['Иваново', 'Фурмановский', 'Комсомольский', 'Тейковский', 'Лежневский', 'Шуйский', 'Родниковский'],
    'Иркутск': ['Иркутский', 'Ангарское', 'Ангарск', 'Шелеховский'],
    'Казань': ['Лаишевский', 'Пестречинский', 'Высокогорский', 'Зеленодольский', 'Верхнеуслонский', 'Новоильмовское'],
    'Калуга': ['Перемышльский', 'Ферзиковский', 'Малоярославецкий', 'Дзержинский', 'Бабынинский'],
    'Кингисеппский': ['Сланцевский', 'Волосовский', 'Ломоносовский', 'Сосновоборский'],
    'Кинельский': ['Кинель', 'Волжский', 'Красноярский', 'Кинель-Черкасский', 'Богатовский', 'Нефтегорский', 'Приволжье'],
    'Кола': ['Североморск', 'Александровск', 'Кольский', 'Кильдинстрой'],
    'Комсомольск-на-Амуре': ['Комсомольский'],
    'Краснодар': ['Динский', 'Красноармейский', 'Тахтамукайский'],
    'Красноярский': ['Володарский', 'Приволжский', 'Наримановский', 'Харабалинский'],
    'Курганинский': ['Лабинский', 'Новокубанский', 'Гулькевичский', 'Тбилисский', 'Шовгеновский', 'Кошехабльский'],
    'Макаровский': ['Долинский', 'Томаринский', 'Углегорский', 'Поронайский', 'Вахрушев'],
    'Мезенский': ['Заполярный', 'Усть-Цилемский', 'Лешуконский', 'Пинежский', 'Приморский'],
    'Михайловский': ['Новомосковск', 'Кимовский', 'Скопинский', 'Пронский', 'Захаровский', 'Серебряные Пруды'],
    'Новороссийск': ['Новороссийский'],
    'Одинцовский': ['Власиха', 'Внуковское', 'Кокошкинское', 'Можайский', 'Красногорск', 'Марушкинское', 'Наро-Фоминский', 'Рузский', 'Истра'],
    'Омск': ['Омский'],
    'Оренбург': ['Оренбургский', 'Сакмарский'],
    'Орёл': ['Орловский'],
    'Петропавловск-Камчатский': ['Вилючинский', 'Елизовский'],
    'Пинежский': ['Лешуконский', 'Мезенский', 'Приморский', 'Холмогорский', 'Виноградовский', 'Верхнетоемский', 'Удорский'],
    'Почепский': ['Трубчевский', 'Погарский', 'Унечский', 'Мглинский', 'Клетнянский', 'Жирятинский', 'Выгоничский', 'Титовское'],
    'Пушкинский': ['Мытищи', 'Дмитровский', 'Сергиево-Посадский', 'Красноармейск', 'Щёлково', 'Восточный', 'Ивантеевка'],
    'Ростов': ['Ростовский'],
    'Саратовский': ['Красноармейский', 'Саратов', 'Лысогорский', 'Татищевский'],
    'Северодвинск': ['Приморский', 'Архангельск'],
    'Северский': ['Туапсинский', 'Геленджик', 'Абинский', 'Горячий Ключ', 'Тахтамукайский', 'Красноармейский'],
    'Тамбов': ['Тамбовский', 'Котовск'],
    'Таштыпский': ['Абаза', 'Таштагольский', 'Турочакский', 'Улаганский', 'Бай-Тайгинский', 'Барун-Хемчикский', 'Сут-Хольский', 'Шушенский', 'Бейский', 'Аскизский'],
    'Тверь': ['Калининский'],
    'Тимашевский': ['Кореновский', 'Динский', 'Калининский', 'Приморско-Ахтарский', 'Брюховецкий'],
    'Тольятти': ['Жигулевск', 'Ставропольский'],
    'Тотемский': ['Бабушкинский', 'Чухломский', 'Солигаличский', 'Междуреченский', 'Сокольский', 'Сямженский', 'Верховажский', 'Тарногский', 'Нюксенский'],
    'Тула': ['Щекинский', 'Дубенский', 'Алексин', 'Ясногорский', 'Веневский', 'Киреевский', 'Ленинский'],
    'Усть-Джегутинский': ['Усть-Джегутинское'],
    'Учалинский': ['Белорецкий', 'Катав-Ивановский', 'Верхнеуральский', 'Уйский', 'Чебаркульский', 'Миасский', 'Златоустовский'],
    'Хасавюрт': ['Хасавюртовский', 'Новолакский', 'Ахвахский'],
    'Целинский': ['Егорлыкский', 'Зерноградский', 'Сальский', 'Песчанокопский', 'Белоглинский'],
    'Чернский': ['Мценский', 'Корсаковский', 'Каменский', 'Тепло-Огаревский', 'Плавский', 'Арсеньевский', 'Славный'],
    'Ярославль': ['Ярославский'],
     "Александровский": ["Петровский", "Благодарненский", "Новоселицкий",
                        "Георгиевский", "Минераловодский", "Андроповский", "Грачёвский"],
    "Петропавловск-Камчатский": ["Вилючинский", "Елизовский"],
    "Советский": ["Новоселицкий", "Будённовский", "Степновский", "Кировский", "Георгиевский"],
    "Ульяновский": ["Ульяновск", "Цильнинский", "Майнский", "Тереньгульский",
                    "Сенгилеевский", "Новоульяновск", "Чердаклинский", "Тетюшский"]
}
     

excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

# Проверка на вырожденные признаки
combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010t+.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Ингушетия нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,ПотреблениеЭ_protest,ПотреблениеВ_protest,Бюджет_protest,...,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,ПотреблениеЭ_event,ПотреблениеВ_event,Бюджет_event,БезДорог_event,Земля_event,Квартиры_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2.79,0.00,400.0,860.0,0.00,0.00,37155.0,...,0.0,0.0,139.0,0.0,0.00,16612.0,0.00,8861.0,418.0,0.144257
1,Хакасия,Таштыпский,95625000,10830.00,0.00,0.0,10.0,600.00,26.10,2401.0,...,0.0,0.0,1600.0,624.0,11.03,1174.0,0.87,661055.0,511.6,1.472160
2,Башкортостан,Альшеевский,80602000,0.80,0.00,200.0,3100.0,723.00,0.00,14205.0,...,0.0,0.0,1322.0,720.0,0.00,-4282.0,0.00,138420.0,419.0,0.164424
3,Башкортостан,Аскинский,80604000,5.10,0.00,0.0,2600.0,1666.00,0.00,-1293.0,...,0.0,200.0,1600.0,1944.0,0.00,-3479.0,8.50,428891.0,352.0,1.308881
4,Башкортостан,Учалинский,80653000,3.50,0.00,0.0,12700.0,1017.10,32.70,-5294.0,...,0.0,0.0,700.0,984.6,0.00,-23553.0,0.10,251184.0,436.0,0.730379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Новосибирская,Новосибирский,50640000,25.90,0.00,600.0,16740.0,0.00,0.00,-5555.0,...,0.0,0.0,1055.0,0.0,0.00,18258.0,0.65,251771.0,0.0,0.250531
93,Омская,Омск,52701000,230.00,2.82,2300.0,15100.0,725.69,19.61,-50359.0,...,0.0,0.0,1400.0,764.9,0.00,-10640.0,0.00,247801.8,437.0,0.569572
94,Камчатский,Петропавловск-Камчатский,30701000,82.10,0.00,0.0,0.0,1469.77,47.80,-275221.0,...,0.0,0.0,0.0,1200.0,5.03,-21725.0,0.00,3510.0,418.0,2.211773
95,Тамбовская,Тамбов,68701000,318.10,0.00,100.0,1000.0,946.50,42.80,-124957.0,...,0.0,350.0,330.0,901.2,6.30,-8137.0,0.00,3547.0,422.0,1.112346


In [19]:
df_2012 = pd.read_csv('прототип_данных_2010t+.csv')
df_2011 = pd.read_csv('без_протестов2008_2010t+.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
]]


from2012.to_csv('связка_махаланобис_2010t+.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event
0,Абакан,95701000,2.79,0.00,400.0,860.0,Саяногорск,95708000,5.6,0.0,0.0,139.0
1,Таштыпский,95625000,10830.00,0.00,0.0,10.0,Орджоникидзевский,95620000,34717.0,0.0,0.0,1600.0
2,Учалинский,80653000,3.50,0.00,0.0,12700.0,Нуримановский,80645000,2.2,0.0,0.0,700.0
3,Благовещенск,80615101,0.00,0.00,0.0,3500.0,Старотуймазинский,80651455,0.0,0.0,0.0,0.0
4,Анапа,3703000,0.90,2.22,300.0,21300.0,Павловский,3639000,18.4,0.0,0.0,5770.0
...,...,...,...,...,...,...,...,...,...,...,...,...
66,Междуреченск,87640153,0.00,0.00,0.0,318.0,Брыкаланск,87604410,0.0,0.0,0.0,0.0
67,Омск,52701000,230.00,2.82,2300.0,15100.0,Москаленский,52632000,7.7,0.0,0.0,1400.0
68,Петропавловск-Камчатский,30701000,82.10,0.00,0.0,0.0,Палана,30851000,8.9,0.0,0.0,0.0
69,Тамбов,68701000,318.10,0.00,100.0,1000.0,Рассказово,68725000,16.2,0.0,350.0,330.0


In [ ]:
df = pd.read_csv('общие_данные_2010t+.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',
         'region_name']]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2010t+.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t+.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event
0,2008,Абакан,95701000,21.10,0.00,381.0,6300.0,Хакасия,Саяногорск,95708000.0,7.53,0.00,0.0,132.0,Хакасия
1,2009,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,Саяногорск,95708000.0,5.60,0.00,0.0,139.0,Хакасия
2,2010,Абакан,95701000,2800.00,0.00,0.0,500.0,Хакасия,Саяногорск,95708000.0,6000.00,0.00,0.0,416.0,Хакасия
3,2011,Абакан,95701000,2100.00,0.00,0.0,1400.0,Хакасия,Саяногорск,95708000.0,6000.00,0.00,0.0,0.0,Хакасия
4,2012,Абакан,95701000,2.10,0.00,180.0,1946.0,Хакасия,Саяногорск,95708000.0,6.00,0.00,0.0,496.9,Хакасия
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,2016,Усть-Джегутинский,91635000,0.00,27.56,0.0,7935.0,Карачаево-Черкесская,Ногайский,91623000.0,0.00,21.70,40.0,580.0,Карачаево-Черкесская
913,2017,Усть-Джегутинский,91635000,0.00,38.92,0.0,5450.0,Карачаево-Черкесская,Ногайский,91623000.0,0.00,26.09,0.0,1500.0,Карачаево-Черкесская
914,2018,Усть-Джегутинский,91635000,0.00,38.92,0.0,250.0,Карачаево-Черкесская,Ногайский,91623000.0,0.00,26.06,0.0,800.0,Карачаево-Черкесская
915,2019,Усть-Джегутинский,91635000,0.00,37.70,0.0,0.0,Карачаево-Черкесская,Ногайский,91623000.0,0.00,26.37,0.0,3770.0,Карачаево-Черкесская
